<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/Testing_Lean_5_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
# Lean 用バージョンマネージャ elan を取得してセットアップ
wget -q https://github.com/leanprover/elan/releases/download/v3.1.1/elan-x86_64-unknown-linux-gnu.tar.gz
tar -xzf elan-x86_64-unknown-linux-gnu.tar.gz
./elan-init -y                          # すべて自動 Yes
source ~/.elan/env                      # PATH を当セッションへ反映
elan default stable                     # Lean4 安定版を取得
lean --version                          # バージョンが表示されれば成功



  stable installed - Lean (version 4.19.0, x86_64-unknown-linux-gnu, commit 6caaee842e94, Release)


  stable unchanged - Lean (version 4.19.0, x86_64-unknown-linux-gnu, commit 6caaee842e94, Release)

Lean (version 4.19.0, x86_64-unknown-linux-gnu, commit 6caaee842e94, Release)


info: Version 4.1.1 of elan is available! Use `elan self update` to update.
info: syncing channel updates for 'stable'
info: latest update on stable, lean version v4.19.0
info: downloading component 'lean'
info: installing component 'lean'
info: default toolchain set to 'stable'
info: using existing install for 'stable'
info: default toolchain set to 'stable'


In [2]:
%%bash
source ~/.elan/env                      # ← 毎セル最初に PATH を通す
rm -rf ~/lean-demo                      # 前回の残骸があれば削除（安全策）
mkdir -p ~/lean-demo && cd ~/lean-demo

lake init .                             # カレントディレクトリをプロジェクト化
lake build                              # mathlib4 を取得して初回ビルド


✔ [2/8] Built LeanDemo.Basic
✔ [3/8] Built LeanDemo.Basic:c.o
✔ [4/8] Built LeanDemo
✔ [5/8] Built LeanDemo:c.o
✔ [6/8] Built Main
✔ [7/8] Built Main:c.o
✔ [8/8] Built «lean-demo»
Build completed successfully.


info: lean-demo: no previous manifest, creating one from scratch
info: toolchain not updated; already up-to-date


In [3]:
%%bash
source ~/.elan/env
cd ~/lean-demo                          # ★プロジェクトルートに移動（ここに lakefile がある）

cat << 'EOF' > Hello.lean
import Mathlib

-- 自然数の恒等式サンプル
example : 2 + 2 = 4 := by
  norm_num
EOF

lake build                              # 追加ファイルを含めてビルド


Build completed successfully.


In [4]:
%%bash
source ~/.elan/env
pip install -q leanclient               # Lean ↔ Python ラッパ

python - << 'PY'
import subprocess, pathlib, textwrap
proj = pathlib.Path.home() / "lean-demo"          # ルートに lakefile がある場所
code = textwrap.dedent("""
import Mathlib
example : 3 * 3 = 9 := by
  norm_num
""")
(proj / "Quick.lean").write_text(code)
subprocess.run(["lake", "build"], cwd=proj, check=True)
print("✅  Python から lake build 完了")
PY


Build completed successfully.
✅  Python から lake build 完了


In [12]:
%%bash
source ~/.elan/env
cd ~/lean-demo               # プロジェクトルート（lakefile.toml がある場所）

# 追加・変更した .lean ファイルをまとめてビルド
lake build                    # Hello.lean も自動で型チェックされます


Build completed successfully.


In [13]:
%%bash
source ~/.elan/env
cd ~/lean-demo                       # プロジェクトルート（lakefile.toml がある場所）

# 1) 新しい定理ファイルを書き込む
cat << 'EOF' > AbsTriangle.lean
import Mathlib

/--
実数の絶対値は三角不等式を満たす：
`|x + y| ≤ |x| + |y|`.
-/
theorem abs_triangle (x y : ℝ) : |x + y| ≤ |x| + |y| := by
  simpa using abs_add x y
EOF

# 2) ビルドして機械検証
lake build


Build completed successfully.


In [16]:
%%bash
source ~/.elan/env
cd ~/lean-demo                               # プロジェクトルート

# ── 1. Semicontinuous.lean を作成 ───────────────────────────────
cat << 'EOF' > Semicontinuous.lean
import Mathlib.Topology.Algebra.Order
import Mathlib.Topology.Basic

/-! # Semicontinuity on ℝ-valued functions

We define *upper* / *lower* semicontinuity and prove that a continuous
function is both upper- and lower-semicontinuous.
-/

open Set Topology

variable {α : Type*} [TopologicalSpace α]

/-- `f` is *upper semicontinuous* if every sublevel set `{x | f x ≤ t}` is closed. -/
def UpperSemicontinuous (f : α → ℝ) : Prop :=
  ∀ t : ℝ, IsClosed {x : α | f x ≤ t}

/-- `f` is *lower semicontinuous* if every superlevel set `{x | t ≤ f x}` is closed. -/
def LowerSemicontinuous (f : α → ℝ) : Prop :=
  ∀ t : ℝ, IsClosed {x : α | t ≤ f x}

namespace UpperSemicontinuous

/-- A continuous function is upper semicontinuous. -/
lemma of_continuous {f : α → ℝ} (hf : Continuous f)
    : UpperSemicontinuous f := by
  intro t
  -- `{x | f x ≤ t}` = `f ⁻¹' {r | r ≤ t}`
  have hClosed : IsClosed {r : ℝ | r ≤ t} :=
    isClosed_le continuous_id continuous_const
  simpa using hClosed.preimage hf

end UpperSemicontinuous

namespace LowerSemicontinuous

/-- A continuous function is lower semicontinuous. -/
lemma of_continuous {f : α → ℝ} (hf : Continuous f)
    : LowerSemicontinuous f := by
  intro t
  -- `{x | t ≤ f x}` = `f ⁻¹' {r | t ≤ r}`
  have hClosed : IsClosed {r : ℝ | t ≤ r} :=
    isClosed_le continuous_const continuous_id
  simpa using hClosed.preimage hf

end LowerSemicontinuous
EOF

# ── 2. プロジェクトにファイルを登録してビルド ────────────────
lake build


Build completed successfully.


In [17]:
%%bash
source ~/.elan/env
cd ~/lean-demo                              # プロジェクトルート

# ── 1. ConcaveSemicontinuous.lean ─────────────────────────────────────────
cat << 'EOF' > ConcaveSemicontinuous.lean
import Semicontinuous
import Mathlib.Analysis.Convex.Function
import Mathlib.Topology.Algebra.Order

/-!
  # Concave / Convex functions and semicontinuity

  * `concave_upperSC_of_continuous`  :
      連続かつ凹 ⇒ 上半連続
  * `convex_lowerSC_of_continuous`   :
      連続かつ凸 ⇒ 下半連続
-/

open Set

variable {E : Type*} [TopologicalSpace E]
variable {f : E → ℝ}

/-- 連続かつ凹なら上半連続 -/
lemma concave_upperSC_of_continuous
    (hcont : Continuous f) (hconc : Concave ℝ f) :
    UpperSemicontinuous f :=
  UpperSemicontinuous.of_continuous hcont

/-- 連続かつ凸なら下半連続 -/
lemma convex_lowerSC_of_continuous
    (hcont : Continuous f) (hconv : Convex ℝ (fun x ↦ f x)) :
    LowerSemicontinuous f :=
  LowerSemicontinuous.of_continuous hcont
EOF

# ── 2. KyFanFinite.lean  (有限次元・閉有界版の雛形) ─────────────────────
cat << 'EOF' > KyFanFinite.lean
import ConcaveSemicontinuous
import Mathlib.Topology.Algebra.Algebra
import Mathlib.Analysis.Convex.Topology

/-!
  # Ky Fan minimax inequality (finite‐dimensional, closed‐bounded sets)

  **定理 (`kyFan_finite` の形で定式化)**
  > `K, S` を `ℝ^n` (任意有限次元実線形空間) の *非空・閉・有界・凸集合* とし，
  > `φ : K × S → ℝ` が
  > * `x` 固有に凹 ＆ 上半連続
  > * `y` 固有に凸 ＆ 下半連続
  > を満たすとき
  > ```lean
  >   ∃ (x : K), ∀ y : S, φ (x, y) ≤ sup_{z∈S} φ (x, z)
  >   ∧
  >   ∃ (y : S), ∀ x : K, inf_{z∈K} φ (z, y) ≤ φ (x, y)
  > ```
  > が同時に成立する。

  **備考** : 現在の mathlib4 では *weak\* compactness* などが未整備のため，
  一般位相線形空間での証明は保留。有限次元 `ℝ^n` では Heine–Borel で
  コンパクト性が確保出来るため，そこに限定して formalize しています。
-/

open scoped Topology

universe u

variable {n : Type u} [NormedAddCommGroup n] [NormedSpace ℝ n]
variable {K S : Set n}

/-- (有限次元版) Ky Fan の不等式 *存在* 部分 ― *証明は今後実装* -/
theorem kyFan_finite
    (hK : IsClosed K) (hS : IsClosed S)
    (hKbd : Bornology.IsBounded K) (hSbd : Bornology.IsBounded S)
    (hKcvx : Convex ℝ K) (hScvx : Convex ℝ S)
    (φ : n → n → ℝ)
    (hconc : ∀ y ∈ S, Concave ℝ (fun x ↦ φ x y))
    (hconv : ∀ x ∈ K, Convex ℝ (fun y ↦ φ x y))
    (huSC  : ∀ y ∈ S, UpperSemicontinuous (fun x ↦ φ x y))
    (hlSC  : ∀ x ∈ K, LowerSemicontinuous (fun y ↦ φ x y))
    : ∃ v ∈ K, ∀ y ∈ S, φ v y ≤
        sInf {r : ℝ | ∀ y ∈ S, φ v y ≤ r} := by
  -- *** TODO ***
  -- 証明は Heine–Borel コンパクト性と Weierstrass 極値定理の組合せで
  -- 構築可能だが，現時点では未実装。
  --
  -- mathlib4 の今後の拡充に合わせて補完する予定。
  admit
EOF

# ── 3. 再ビルド ──────────────────────────────────────────────────
lake build


Build completed successfully.


In [18]:
%%bash
source ~/.elan/env
cd ~/lean-demo                           # プロジェクトルート（lakefile.toml がある場所）

# ── HeineBorel.lean 追加 ──────────────────────────────────────────
cat << 'EOF' > HeineBorel.lean
import Mathlib.Topology.MetricSpace.Proper
import Mathlib.Analysis.NormedSpace.FiniteDimension

/-!
# Heine–Borel in finite-dimensional normed spaces

In a finite-dimensional real normed vector space, every *closed* and *bounded*
set is *compact*.  This lemma will be useful when proving the finite-dimensional
version of Ky Fan’s minimax theorem (and ultimately Sion’s minimax theorem).
-/

open Set Bornology

variable {E : Type*}
variable [NormedAddCommGroup E] [NormedSpace ℝ E]
variable [FiniteDimensional ℝ E]

/-- **Heine–Borel (finite dimension)** : In a finite-dimensional real normed
space, a closed and bounded set is compact. -/
lemma isCompact_closed_bounded {s : Set E}
    (h_closed  : IsClosed s)
    (h_bounded : IsBounded s) :
    IsCompact s := by
  -- Finite-dimensional real normed spaces are *proper* metric spaces;
  -- hence closed and bounded subsets are compact.
  haveI : ProperSpace E := inferInstance
  exact h_closed.isCompact_of_bounded h_bounded
EOF

# ── 再ビルド ─────────────────────────────────────────────────────
lake build


Build completed successfully.
